<a href="https://colab.research.google.com/github/AD6969/Liquity_Internship_Aditya_Dhotre/blob/main/Chat_Bot_Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [6]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [7]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [8]:
GROQ_API_KEY = "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"

In [9]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [10]:
from langchain.document_loaders import UnstructuredPDFLoader

# Specify the path to your PDF file directly
file_path = "/content/Compendium-2023-merged_repaired.pdf"

# Create an instance of the UnstructuredPDFLoader
loader = UnstructuredPDFLoader(file_path)

# Load the document
documents = loader.load()


In [11]:
print(documents)

[Document(metadata={'source': '/content/Compendium-2023-merged_repaired.pdf'}, page_content='Preface\n\nThe Right to Information Act, 2005 was promulgated to promote transparency and accountability in the working of every public authority. A thorough knowledge of the provisions of the Act on the part of the Public Information Officers/First Appellate Authorities of the public authorities is key to the successful implementation of the Act.\n\nDuring the course of evolving of the RTI Act, 2005, Department of Personnel and Training (DoPT) has issued clarifications on various provisions of the Act. For the convenience of the citizens, a compendium of Office Memorandums (OMs) and Notifications issued by DoPT has been launched online, with topic-wise search facility.\n\nI hope that the updated Compendium would further enhance the understanding of the provisions of the Act by both information seekers and information providers. This would go a long way in further strengthening the RTl Act.\n\n

In [12]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [13]:
persist_directory = "doc_db"

In [14]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [15]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [16]:
retriever = vectorstore.as_retriever()

In [17]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [19]:
query = "What is The Right to Information Act, 2005"
response = qa_chain.invoke({"query":query})

In [20]:
print(response)

{'query': 'What is The Right to Information Act, 2005', 'result': 'The Right to Information Act, 2005 is a law enacted by the Parliament of India to provide citizens with the right to access information from the government and its various departments. The Act was passed on June 15, 2005, and it came into force on October 12, 2005.\n\nThe main objective of the Act is to promote transparency and accountability in the functioning of the government by making information available to the citizens. The Act provides for the establishment of a Central Information Commission and State Information Commissions to oversee the implementation of the Act and to handle complaints related to the denial of information.\n\nKey features of the Act include:\n\n1. Right to Information: The Act gives citizens the right to access information from the government and its departments, subject to certain exceptions.\n2. Public Authorities: The Act defines public authorities as any government department, ministry,

In [21]:
print(response["result"])

The Right to Information Act, 2005 is a law enacted by the Parliament of India to provide citizens with the right to access information from the government and its various departments. The Act was passed on June 15, 2005, and it came into force on October 12, 2005.

The main objective of the Act is to promote transparency and accountability in the functioning of the government by making information available to the citizens. The Act provides for the establishment of a Central Information Commission and State Information Commissions to oversee the implementation of the Act and to handle complaints related to the denial of information.

Key features of the Act include:

1. Right to Information: The Act gives citizens the right to access information from the government and its departments, subject to certain exceptions.
2. Public Authorities: The Act defines public authorities as any government department, ministry, or organization that is funded by the government.
3. Information Officers

In [22]:
query = "Who is responsible for designating Public Information Officers (PIOs) under the RTI Act, 2005?"
response = qa_chain.invoke({"query":query})
print(response["result"])

According to the given context, every public authority is responsible for designating Public Information Officers (PIOs) under the RTI Act, 2005. Specifically, sub-section (1) of section 5 of the Act states that "every public authority shall designate Public Information Officers in all administrative units or offices under it within one hundred days of the enactment of the Act."


In [23]:
print(response["result"])

According to the given context, every public authority is responsible for designating Public Information Officers (PIOs) under the RTI Act, 2005. Specifically, sub-section (1) of section 5 of the Act states that "every public authority shall designate Public Information Officers in all administrative units or offices under it within one hundred days of the enactment of the Act."


In [ ]:
# Assuming qa_chain is already initialized and ready to use.

# Loop to ask multiple questions from the user
while True:
    # Take the user's question as input
    query = input("Enter your question (or type 'exit' to quit): ")

    # Check if the user wants to exit the loop
    if query.lower() == "exit":
        print("Exiting the question loop.")
        break

    # Get the response from the QA system
    response = qa_chain.invoke({"query": query})

    # Print the result
    print("Question:", response["query"])
    print("Answer:", response["result"])


Enter your question (or type 'exit' to quit): What is the main purpose of the Right to Information Act, 2005?
Question: What is the main purpose of the Right to Information Act, 2005?
Answer: The main purpose of the Right to Information Act, 2005 is to empower citizens to be aware of the actions taken by state agencies and governments, thereby promoting transparency and accountability in governance. It aims to provide citizens with access to information in the possession of state agencies and governments, enabling them to make informed decisions and hold those in power accountable for their actions.
Enter your question (or type 'exit' to quit): Who is responsible for designating Public Information Officers (PIOs) under the RTI Act, 2005?
Question: Who is responsible for designating Public Information Officers (PIOs) under the RTI Act, 2005?
Answer: According to the given context, every public authority is responsible for designating Public Information Officers (PIOs) under the RTI Act,